# [Chainer] Azure MLで、自分のファイルを使って、ハイパーパラメーターの調整まで

先ほどの解説では用意されてものを使用してきました。  
このハンズオンでは自身のデータを用いて、ハンズオン終了後にもAzure MLを使いこなすようになることを目指します。 また、Chainer を使用して学習を行います。

このハンズオンでは下記の内容を行います。   
- モデルのトレーニング
- ハイパーパラメーターの調整
- (開発途中)デプロイ

In [ ]:
import azureml.core

print("This notebook was created using version 1.0.15 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

## Azure Machine Services の Workspace へのログインと接続

Azure Machine Learning service へ接続を行います。アクセス権設定は、Azure Machine Learning services 作成時に自動的に行われています。作成した方はもちろん、その Workspace へアクセスできます。

Azure Notebook からは、シングルサインオンができます。以下のセルを実行すると、ログイン用のダイアログが表示されます。そのままログインをして先に進んでください。

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## コンピューティング ターゲットの設定

[コンピューティングターゲット](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target)を作成する必要があります。このチュートリアルではAzure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute))を使用します。  
（コンピューティングターゲットは計算を実行する場所を決定するようなイメージです。）

※AmlComputeの作成には約5分かかります。  
その名前のAmlComputeが既にワークスペースにある場合、このコードは作成プロセスをスキップします。

他のAzureサービスと同様に、Azure Machine Learningサービスに関連する特定のリソース（AmlComputeなど）には制限があります。  
デフォルトの制限と、より多くのクォータを要求する方法についての[この記事](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas)を読んでください。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_NC6s_v3", ## Standard_NC6
                                                       min_nodes=1,
                                                       max_nodes=4) ## vm_priority='lowpriority' | `dedicated'

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

## (Option) Data Science VM の設定

AmlCompute の作成ができなかった方向けです。AmlCompute の作成ができた方は **スキップ** してください。
実行すると、AmlCompute設定が上書きされてしまいます。

Azure 無償トライアルなどで、GPUインスタンスのクォータ引き上げが出来ない場合もあります。
その場合、Data Science VM など単独のGPU仮想マシンへ接続して学習環境として利用できます。

[こちら](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-set-up-training-targets#vm) のドキュメントを参照して、*事前*に Data Science VM を作成してから、以下を実行してください。

In [ ]:
from azureml.core.compute import RemoteCompute, ComputeTarget

# Create the compute config 
compute_target_name = "attach-dsvm"
attach_config = RemoteCompute.attach_configuration(address = "<fqdn もしくは IP Address>",
                                                 ssh_port=22,
                                                 username='<username>',
                                                 password="<password>")

# Attach the compute
compute = ComputeTarget.attach(ws, compute_target_name, attach_config)

compute.wait_for_completion(show_output=True)

## データストアにデータをUpload

リモートコンピューティングクラスタを使用して学習する準備が整いました。  
まずは学習用のデータを準備します。  
今回は手持ちのデータを使用するため、データを[データストア](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data)にUploadする必要があります。  
データストアは、データを格納できる場所であり、データをマウントするか、計算対象にデータをコピーすることによってRunにアクセスできるようになります。  
データストアは、Azure Blob StorageまたはAzure File Shareのいずれかでバックアップできます（ADLSは将来サポートされる予定です）。  
単純なデータ処理のために、データがまだBlob StorageまたはFile Shareにない場合は、各ワークスペースに使用可能なデフォルトのデータストアが用意されています。

### プロジェクトディレクトリの作成

学習実行に必要なコードを格納するディレクトリを作成します。  
このディレクトリには学習を実行するコードと、それに依存関係のファイルなどを格納するする必要があります。  

In [ ]:
import os

project_folder = './chainer-doc_cat'
os.makedirs(project_folder, exist_ok=True)

今回は`chainer-doc_cat`という名前のプロジェクトフォルダを作成しました。  

### データストアの準備

今回の問題設定は犬と猫の分類です。  
先ほどはWeb上に準備されたデータを取得して学習を行いましたが、今回は自身のデータを使用することを想定して学習を行います。  
今回はデフォルトで設定されているデータストアを使用します。  

Github からクローンした際に、`/data/doc_cat` フォルダーに、学習用のデータセットも一緒にコピーしています。

In [ ]:
datastores = ws.datastores
for name, dss in datastores.items():
    print(name, dss.datastore_type)
    
ds = ws.get_default_datastore()
print('default: ' + ds.name, ds.datastore_type)

この次のステップでは、トレーニングとテストセットをワークスペースのデフォルトデータストアにアップロードします。  
これは後で`AmlCompute`クラスターにマウントしてトレーニングを行います。

In [ ]:
%%time
ds.upload(src_dir='./data', target_path='data', overwrite=True, show_progress=True)

## 学習スクリプトの作成

基本的には先ほど使用したものを用います。  
データの読み込みの部分が持っているデータを扱うため、データをダウンロードする必要がなくなります。  

スクリプトは下記のディレクトリに格納しておきます。  

In [ ]:
%%writefile chainer-doc_cat/train.py

import numpy as np
import chainer
import chainer.links as L
import chainer.functions as F
import cupy
from glob import glob
from chainer.datasets import TransformDataset
from zipfile import ZipFile
import time
import os
import copy
import argparse
import random
from model import GoogleNet

import chainermn

from azureml.core.run import Run
# get the Azure ML run object
run = Run.get_context()

# データの読み込み関数
def get_data(data_folder):
    data_file = os.path.join(data_folder, 'data/dog-cat.zip')
#    data_file = data_folder
    
    print('training dataset is stored here:', data_file)

    # extract files
    with ZipFile(data_file, 'r') as zipfile:
        try:
            print('extracting files...')
            zipfile.extractall()
            print('finished extracting')
            data_dir = zipfile.namelist()[0]
        except:
            data_dir = zipfile.namelist()[0]

    return data_dir

# データに対する前処理用の関数
def transform(inputs):
    img, label = inputs
    img = img.transpose(1,2,0)
    img = L.model.vision.googlenet.prepare(img)
    img, label = cupy.asarray(img), cupy.asarray(label)
    return img, label

# データセットの作成と前処理の適応
def prepare_data(data_dir):
    labels = {'dog':0, 'cat':1}
    # パスの取得
    train_dog = [(path, labels['dog']) for path in glob('{}{}/{}/*'.format(data_dir, 'train','dog'))]
    train_cat = [(path, labels['cat']) for path in glob('{}{}/{}/*'.format(data_dir, 'train','cat'))]
    val_dog = [(path, labels['dog']) for path in glob('{}{}/{}/*'.format(data_dir, 'val','dog'))]
    val_cat = [(path, labels['cat']) for path in glob('{}{}/{}/*'.format(data_dir, 'val','cat'))]
    # データセットの作成
    train = chainer.datasets.LabeledImageDataset(train_dog+train_cat)
    valid= chainer.datasets.LabeledImageDataset(val_dog+val_cat)
    # 前処理の適応
    train = TransformDataset(train, transform)
    valid = TransformDataset(valid, transform)

    return train, valid

# 乱数のシード固定用の関数
def reset_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    if chainer.cuda.available:
        chainer.cuda.cupy.random.seed(seed)

# 学習用の関数
def train_model(model, comm, optimizer, data_dir, batchsize, num_epochs, learning_rate, lr_shift_timing, lr_shift_rate):
    train, valid = prepare_data(data_dir)
    train = chainermn.scatter_dataset(train, comm, shuffle=True)
    valid = chainermn.scatter_dataset(valid, comm, shuffle=True)
    # Iteratorの定義
    train_iter = chainer.iterators.SerialIterator(train, batchsize)
    valid_iter = chainer.iterators.SerialIterator(valid, batchsize, repeat=False, shuffle=False)
    train_iter.reset()
    valid_iter.reset()
    best_accuracy = 0
    print("starting training")

    # 学習ループの実行
    while train_iter.epoch < num_epochs:

        #　------------  学習の1イテレーション  ------------

      # データの取得
        train_batch = train_iter.next()
        x_train, t_train = chainer.dataset.concat_examples(train_batch)

      # 予測値の計算
        y_train = model(x_train)

      # ロスの計算
        loss_train = F.softmax_cross_entropy(y_train, t_train)

      # 勾配の計算
        model.cleargrads()
        loss_train.backward()

      # パラメータの更新
        optimizer.update()

      # 検証データで精度を計算
        accuracy_train = F.accuracy(y_train, t_train)
        accuracy_train = float(accuracy_train.data)

        #  ----------------  ここまで  ----------------

        if train_iter.is_new_epoch: # 新しいエポックに入った時のみ計算

            valid_accuracies, valid_losses = [], []
            while True:
                # 検証データの取得
                valid_batch = valid_iter.next()
                x_valid, t_valid = chainer.dataset.concat_examples(valid_batch)

             # 検証用データで順伝播の計算を実行
                with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
                    y_valid = model(x_valid)

              # 検証データで損失関数を計算
                loss_valid = F.softmax_cross_entropy(y_valid, t_valid)
                valid_losses.append(float(loss_valid.data))

              # 検証データで精度を計算
                accuracy_valid = F.accuracy(y_valid, t_valid)
                valid_accuracies.append(float(accuracy_valid.data))

                if valid_iter.is_new_epoch: # 追加：1エポック計算し終わると、イテレーターをリセット
                    valid_iter.reset()
                    break

            # Exponentialの適応
            if (train_iter.epoch) % lr_shift_timing == 0:
                try:
                    optimizer.alpha = optimizer.alpha * lr_shift_rate
                    print(optimizer.alpha)
                except:
                    optimizer.lr = optimizer.lr * lr_shift_rate
                    print(optimizer.lr)
                    
         # 結果を表示
            run.log('best_val_acc', np.mean(valid_accuracies))
            print('valid_loss:{:.04f} valid_accuracy:{:.04f}'.format(np.mean(valid_losses), np.mean(valid_accuracies)))
            print('---')
            
    return model

# 学習を実行する関数
def fine_tune_model(device, comm, num_epochs, data_dir, learning_rate, optimizer_name, batchsize, lr_shift_timing, lr_shift_rate):
    # ハイパーパラメータの観測
    run.log('lr', np.float(learning_rate))
    run.log('opt_name', str(optimizer_name))
    run.log('batchsize', int(batchsize))

    # インスタンス化
    reset_seed(0)
    model = GoogleNet()
    if device >= 0:
        chainer.cuda.get_device_from_id(device).use()
        model.to_gpu(device)
        print("1, passed the model to gpu")
    else:
        print("2, didn't passe the model to gpu")
    # Optimizerの定義とmodelとの紐づけ
    if 'SGD' == optimizer_name:
        optimizer = chainer.optimizers.SGD(lr=learning_rate)
    elif 'MomentumSGD' == optimizer_name:
        optimizer = chainer.optimizers.MomentumSGD(lr=learning_rate)
    else:
        optimizer = chainer.optimizers.Adam(alpha=learning_rate)
    optimizer = chainermn.create_multi_node_optimizer(optimizer, comm).setup(model)
    model.base.disable_update() # 学習済みモデル部分の学習を行わない
    # 学習の実行
    model = train_model(model, comm, optimizer, data_dir, batchsize, num_epochs, learning_rate, lr_shift_timing, lr_shift_rate)
    return model

def main():
    print("Chainer version:", chainer.__version__)

    # get command-line arguments
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_folder', type=str, dest='data_folder', help='data folder mounting point')
    parser.add_argument('--num_epochs', type=int, default=20,help='number of epochs to train')
    parser.add_argument('--output_dir', type=str, help='output directory')
    parser.add_argument('--communicator', type=str, default='hierarchical', help='Type of communicator')
    parser.add_argument('--learning_rate', type=float,default=0.01, help='learning rate')
    parser.add_argument('--optimizer_name', type=str,default='SGD', help='name of optimizer')
    parser.add_argument('--batchsize', type=int, default=64, help='batchsize')
    parser.add_argument('--lr_shift_timing', type=int, default=6, help='Exponential shigt timing by epoch')
    parser.add_argument('--lr_shift_rate', type=float, default=0.01, help='Exponential shigt ratio by epoch')
    args = parser.parse_args()
    
    comm = chainermn.create_communicator(args.communicator)
    device = comm.intra_rank
    print("communicator is {}, device is{}".format(comm, device))
    
    print(" insidedata_folder: " + args.data_folder)
    data_dir = get_data(args.data_folder)
    print("data directory is: " + data_dir)
    model = fine_tune_model(device, comm, args.num_epochs, data_dir, args.learning_rate, args.optimizer_name, args.batchsize, args.lr_shift_timing, args.lr_shift_rate)
    os.makedirs(args.output_dir, exist_ok=True)
    model.to_cpu()
    chainer.serializers.save_npz(os.path.join(args.output_dir, 'model.npz'), model)

if __name__ == "__main__":
    main()

モデルを定義したスクリプトを準備しておきます。  

In [ ]:
%%writefile chainer-doc_cat/model.py

import numpy as np
import chainer
import chainer.links as L
import chainer.functions as F

class GoogleNet(chainer.Chain):

    def __init__(self, n_out=2):
        super(GoogleNet, self).__init__()
        with self.init_scope():
            self.base = L.GoogLeNet()
            self.fc = L.Linear(None, n_out)

    def forward(self, x):
        h = self.base(x, layers=['pool5'])
        h = self.fc(h['pool5'])
        return h

### Experimentの作成
ワークスペースですべての実行結果を追跡するために[Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) を作成します。  

In [ ]:
from azureml.core import Experiment

experiment_name = 'chainer-dog-cat'
experiment = Experiment(ws, name=experiment_name)

## Chainer estimatorの作成

Azure ML SDKの Chainer estimatorを使用すると、単一ノードと分散の両方の実行について、Chainerトレーニングジョブを簡単に送信できます。   
Chainer estimatorの詳細については、[こちら](https://docs.microsoft.com/ja-jp/python/api/azureml-train-core/azureml.train.dnn.chainer?view=azure-ml-py)を参照してください。次のコードは単一ノードのChainerジョブを定義します。

また、DataStore を学習スクリプトからアクセスさせるための詳細については[こちら](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-access-data)を参照してください。

先ほどの PyTouch の Estimater を呼び出すコードと、ほとんど差がない点を確認します。

In [ ]:
from azureml.train.estimator import *
from azureml.train.dnn import *

script_params = {
    '--data_folder': ds.as_mount(),
    '--num_epochs': 25,
    '--output_dir': './outputs',
    '--communicator': 'non_cuda_aware'
}

estimator = Chainer(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='train.py',
                      distributed_backend='mpi',     
                      pip_packages=['cupy-cuda90', 'mpi4py', 'cython', 'chainer==5.1', 'chainermn', 'chainercv'],
                      use_gpu=True)

## 学習の実行

下記のコードで学習を実行します。

In [ ]:
run = experiment.submit(estimator)
print(run)

### 学習の経過の確認

学習の実行には下記の4ステップがあります。  

1. 準備：Chainer Estimater で指定されたPython環境に合わせてdockerイメージが作成され、それがワークスペースのAzure Container Registryにアップロードされます。このステップはPython環境ごとに一度だけ起こります。（その後の実行のためにコンテナはキャッシュされます。）画像の作成とアップロードには約5分かかります。ジョブの準備中、ログは実行履歴にストリーミングされ、イメージ作成の進行状況を監視するために表示できます。

2. スケーリング：計算をスケールアップする必要がある場合（つまり、バッチAIクラスターで現在実行可能な数より多くのノードを実行する必要がある場合）、クラスターは必要な数のノードを使用可能にするためにスケールアップを試みます。スケーリングは通常約5分かかります。

3. 実行中：スクリプトフォルダ内のすべてのスクリプトがコンピューティングターゲットにアップロードされ、データストアがマウントまたはコピーされてentry_scriptが実行されます。ジョブの実行中は、stdoutと./logsフォルダが実行履歴にストリーミングされ、実行の進行状況を監視するために表示できます。

4. 後処理：実行の./outputsフォルダが実行履歴にコピーされます。

In [ ]:
# to get more details of your run
print(run.get_details())

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)

手持ちのデータを用いて学習を実行することができました。  
ですが、現在はGPUマシンを1台立てて、学習を行っただけです。  

続いてはGPUマシンを複数使用して、ハイパーパラメータの自動チューニングを確認します。  

## ハイパーパラメーターのチューニング

基本的に必要な設定は下記になります。  

- ハイパーパラメータを探索する範囲を選択
- ハイパーパラメータを探索する方法を選択（ランダムスイープ、グリッドサーチ、ベイズ最適化）
- 最大化or最小化したいメトリックの指定

詳細の設定方法については[こちらの公式ドキュメント](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-tune-hyperparameters)を確認してください。  


### 学習率の調整

トレーニングスクリプト（`train.py`）は数エポックごとに学習率を減少させるために学習率スケジュールを使うので、最初の学習率とmomentumのパラメータを調整しましょう。  
今回はランダムスイープを使用します。  
（この節でてくる語句については[公式ドキュメント](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/how-to-access-data)を確認してください。）


In [ ]:
from azureml.train.hyperdrive import *

param_sampling = RandomParameterSampling( {
    'learning_rate':uniform(0.005, 0.01),
    'optimizer_name': choice('SGD', 'MomentumSGD', 'Adam'),
    'batchsize': choice(16, 32, 64),
    }
)

### 早期終了(Early Stopping)の適応

次に、パフォーマンスの低い実行を早期終了するための設定を行います。  

ここでは `BanditPolicy`を使います。これは主要なすべての実行を終了します。  
設定はシンプルに3つの要素を指定します。

- slack_factor：早期終了を行う閾値（指定したメトリックの最高のものとの誤差）
- evaluation_interval：早期終了を確認する周期（エポック数）
- delay_evaluation：早期終了を確認を始めるタイミング（0エポック目から何エポック遅らせるか）

ここでは、ポリシーをエポックごとに適用します（エポックごとに `best_val_acc`メトリックをレポートし、` evaluation_interval = 1`になるため）  
最初の `10`エポック（` delay_evaluation = 10`）の後まで最初のポリシー評価を遅らせることに注意してください。
BanditPolicyおよびその他のポリシーの詳細については、[ここ](https://docs.microsoft.com/azure/machine-learning/service/how-to-tune-hyperparameters#specify-an-early-termination-policy)を参照してください。

In [ ]:
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

### 設定の定義

設定には`HyperDriveRunConfig()`を使用します。  
詳細は[こちらの公式ドキュメント](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriverunconfig?view=azure-ml-py)を確認してください。  

`primary_metric_name='best_val_acc'`で検証データへの精度を評価指標として使用する事を決定し、`primary_metric_goal=PrimaryMetricGoal.MAXIMIZE`でその値が最大となる事を目指すと設定しています。  
`MINIMIZE`と指定することによって、最小にする事を目指すことも可能です。  

また`max_total_runs=4`で学習実行数を指定、`max_concurrent_runs=8`で同時に学習を行う数を指定しています。

In [ ]:
hyperdrive_run_config = HyperDriveRunConfig(estimator=estimator,
                                            hyperparameter_sampling=param_sampling, 
                                            policy=early_termination_policy,
                                            primary_metric_name='best_val_acc',
                                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs=8,
                                            max_concurrent_runs=4)

これで設定が完了したので、早速学習を行ってみましょう。

学習時の環境にもよりますが、並列度 4で、**10分程度**かかります。

初期設定の通り、並列度 4、最大8、AmlCompute 4の場合、20-25分程度かかります。

In [ ]:
# HyperDrive runの開始
hyperdrive_run = experiment.submit(hyperdrive_run_config)

In [ ]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

### 最適なモデルの取得



すべての実行が完了すると、モデルを最も正確に生成した実行を見つけることができます。

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metribcs()
print(best_run)

最後に、最も優れた実行結果からモデルをワークスペースに登録します。   
`model_path`パラメータはあなたの` output`ディレクトリのモデルファイルへのリモートVM上の相対パスを取ります。次のセクションでは、この登録モデルをWebサービスとしてデプロイします。

## 最適なモデルのデプロイ

In [ ]:
model = best_run.register_model(model_name = 'Chainer-dogcat', model_path = 'outputs/model.pt')
print(model.name, model.id, model.version, sep = '\t')

# 以降、開発中!!!

ここ以降まだ確認できていません。

### スコアリングスクリプトの作成

- `init（）`：この関数では、通常モデルを `global`オブジェクトにロードします。この関数はDockerコンテナが起動されたときに一度だけ実行されます。
- `run（input_data）`：この関数では、新たな入力データ対して学習済みモデルを使用して推論を実行します。通常は入力と出力は通常シリアライゼーションとデシリアライゼーションのフォーマットとしてJSONを使用しますが、他のフォーマットも使用することが可能です。

In [ ]:
%%writefile chainer-doc_cat/score.py

import torch
import torch.nn as nn
from torchvision import transforms
import json

from azureml.core.model import Model


def init():
    global model
    model_path = Model.get_model_path('pytorch-dogcat')
    model = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.eval()


def run(input_data):
    input_data = torch.tensor(json.loads(input_data)['data'])

    # get prediction
    with torch.no_grad():
        output = model(input_data)
        classes = ['cat', 'dog']
        softmax = nn.Softmax(dim=1)
        pred_probs = softmax(output).numpy()[0]
        index = torch.argmax(output, 1)

    result = {"label": classes[index], "probability": str(pred_probs[index])}
    return result

### 環境ファイルを作成する

スコアリングスクリプトのすべてのパッケージ依存関係を指定する環境ファイル（ `myenv.yml`）を作成する必要があります。このファイルは、Azure MLによってこれらのすべての依存関係がDockerイメージにインストールされるようにするために使用されます。この場合、 `azureml-core`、` torch`、そして `torchvision`が必要になります。

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=['azureml-defaults', 'torch', 'torchvision'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

### Dockerイメージの設定

ACIコンテナーを構築するために使用するDockerイメージを構成します。  
詳細については[こちらの公式ドキュメント](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.image.containerimage?view=azure-ml-py)を確認してください。  

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script='pytorch_score.py', 
                                                  runtime='python', 
                                                  conda_file='myenv.yml',
                                                  description='Image with hymenoptera model')

### ACIコンテナの設定

デプロイのための準備がほぼ整いました。   
ACIコンテナに必要なCPUの数とギガバイトのRAMを指定するためのデプロイメント構成ファイルを作成します。  
それは作成したモデルに依存しますが、一般的なモデルではデフォルトの `1`コアと` 1`ギガバイトのRAMで十分なケースが多いです。  

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={'data': 'dog_cat',  'method':'transfer learning', 'framework':'chainer'},
                                               description='Classify dogs/cats using transfer learning with Chainer')

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={'data': 'dogcat',  'method':'transfer learning', 'framework':'pytorch'},
                                               description='Classify ants/bees using transfer learning with PyTorch')

### Container Instances にデプロイする

最後に、登録したモデルからWebサービスをデプロイしましょう。  
前の手順で作成したACI設定ファイルとイメージ設定ファイルを使用してWebサービスをデプロイします。  

リストの中の `model`オブジェクトを` models`パラメータに渡します。  
複数の登録済みモデルをデプロイする場合は、このリストに他のモデルを追加してください。　　

In [ ]:
%%time
from azureml.core.webservice import Webservice

service_name = 'classify-dog-cat'
service = Webservice.deploy_from_model(workspace=ws,
                                       name=service_name,
                                       models=[model],
                                       image_config=image_config,
                                       deployment_config=aciconfig,)

service.wait_for_deployment(show_output=True)
print(service.state)

通常デプロイには7~8分かかります。  
下記のように表示されればデプロイが成功しています。  

```
SucceededACI service creation operation finished, operation "Succeeded"

```

#### デプロイがうまくいかない場合

もし、何らかの理由でデプロイが失敗して再デプロイする必要がある場合は、必ずサービスを`service.delete（）`で削除してください。  

**また、デプロイに問題が発生した場合、まず下記のコマンドを実行して、サービスからログを取得しましょう。**  

`service.get_logs()`

## デプロイされたサービスをテストする

最後に、デプロイしたWebサービスをテストしましょう。  
データをJSON文字列としてACIでホストされているWebサービスに送信し、SDKの `run` APIを使用してサービスを呼び出します。  
ここで、検証データからイメージを取得して推論を実行します。

In [ ]:
import os, json
from PIL import Image
import matplotlib.pyplot as plt

plt.imshow(Image.open('test_img2.jpeg'))

画像データに対して学習時と同じ前処理を適応し、推論が実行できる状態に変更します。  

In [ ]:
import torch
from torchvision import transforms
    
def preprocess(image_file):
    """Preprocess the input image."""
    data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_file)
    image = data_transforms(image).float()
    image = torch.tensor(image)
    image = image.unsqueeze(0)
    return image.numpy()

In [ ]:
input_data = preprocess('test_img2.jpeg')
result = service.run(input_data=json.dumps({'data': input_data.tolist()}))
print(result)

うまく推論できていることが確認できました。  
これで自身のデータに対してもAzure MLを用いて、GPUクラスタを用いてのハイパーパラメータの調整、そしてデプロイまでを行うことのできる力がつきました。

## 後片付け

Webサービスが不要になったら、API呼び出しで簡単に削除しておきましょう。

In [ ]:
service.delete()

これでハンズオンは終了です。  
皆さんお疲れ様でした。